## Import Library

In [13]:
import pandas as pd
import re

## Load data

In [14]:
file_path = r'C:\Users\user\CAPSTONE-PROJEK---CODEY\Machine Learning\kategori soal\function.csv'

data = pd.read_csv(file_path)

data.head()

,Instruction,Input,Output,materi,question_id
0,Generate a function in Python to convert a giv...,17,def convertToBinary(n): \n \n if n > 1: \n...,Function,14
1,Create a function in Python to compute a Fibon...,10,"def fibonacci(n): \n if n < 0: \n print(""Inco...",Function,18
2,Construct a function that will find the value ...,"dictionary = {'key1':1, 'key2':2, 'key3':3}\nk...","def find_value(dictionary, key):\n value = ...",Function,21
3,Design a Python function that converts strings...,This is an example sentence,def convertToCapitalize(sentence)\n words =...,Function,25
4,Create an AWS Lambda function in Python that a...,"{\n ""x"": 10,\n ""y"": 20\n}","import json\n\ndef handler(event, context):\n ...",Function,52


In [15]:
def add_placeholder(code):
    """
    Gantikan bagian tertentu dalam kode dengan placeholder '___'
    """
    # Pola regex untuk mencari kata kunci dan mengganti kata setelahnya dengan placeholder
    patterns = [
         r'\b(def|function|return)\s+(\w+)\b\b'
    ]
    
    # Cari kecocokan pertama dalam seluruh kode
    for pattern in patterns:
        match = re.search(pattern, code)
        if match:
            # Ganti kata setelah kata kunci dengan placeholder '___' pada kecocokan pertama
            code = re.sub(pattern, r'\1 ___', code, count=1)  # Hanya mengganti kecocokan pertama
    
    return code


Menambah kolom Template untuk soal 

In [16]:
# Terapkan fungsi pada kolom 'Full Code'
data['Template'] = data['Output'].apply(add_placeholder)
print(data['Template'][0])
print(data['Output'][0])
print(data['Input'][0])

def ___(n): 
  
    if n > 1: 
        convertToBinary(n//2) 
  
    print(n % 2,end = '') 
  

# driver code 
num = 17
convertToBinary(num)
def convertToBinary(n): 
  
    if n > 1: 
        convertToBinary(n//2) 
  
    print(n % 2,end = '') 
  

# driver code 
num = 17
convertToBinary(num)
17


Membuat fungsi untuk mengecek data soal memiliki placeholder

In [17]:
def check_placeholder_in_column(df, column_name, placeholder="___"):
    """
    Cek apakah kolom tertentu dalam DataFrame mengandung placeholder.
    
    Args:
        df (pd.DataFrame): DataFrame yang akan dicek.
        column_name (str): Nama kolom yang akan diperiksa.
        placeholder (str): Placeholder yang dicari (default: "___").
    
    Returns:
        pd.Series: Kolom boolean (True jika ada placeholder, False jika tidak).
    """
    return df[column_name].str.contains(placeholder, na=False)


In [18]:
data["has_placeholder"] = check_placeholder_in_column(data, "Template")

print(data["has_placeholder"].value_counts())

has_placeholder
True     495
False     17
Name: count, dtype: int64


In [19]:
# Hapus baris dengan nilai False
data = data[data["has_placeholder"]]

print(data["has_placeholder"].value_counts())

# Hapus kolom 'has_placeholder' jika tidak diperlukan lagi
data = data.drop(columns=["has_placeholder"])


has_placeholder
True    495
Name: count, dtype: int64


In [20]:
def extract_keyword_variable(code):
    """
    Mengambil kata setelah kata kunci seperti def, function, dll dan mengembalikannya.
    """
    # Pola regex untuk mencari kata kunci dan mengekstrak kata setelahnya
    pattern = r'\b(def|function|return)\s+(\w+)\b\b'
    
    match = re.search(pattern, code)
    
    # Jika ada kecocokan, ambil kata setelah kata kunci (hasil kedua dalam tuple)
    if match:
        return match[2]  # Kata setelah kata kunci
    
    return None

# Terapkan fungsi untuk membuat kolom baru
data['correct_answer'] = data['Output'].apply(extract_keyword_variable)



In [21]:
print(data['correct_answer'][12])
print(data['Template'][12])

add_arrays
def ___(arr1, arr2):
 if len(arr1) != len(arr2):
  raise Exception("Arrays are not of the same size!")
 return [x + y for x, y in zip(arr1, arr2)]

add_arrays(arr1, arr2)  # [4, 6, 8]


## Save Data

In [22]:
# Simpan dataset yang telah diproses
data.to_csv('Function_with_placeholders.csv', index=False)

In [24]:
import json

# Membuat struktur JSON
json_data = {"quiz": []}

for _, row in data.iterrows():
    question_id = row['question_id']
    material = row['materi']
    instruction = row['Instruction']
    question = row['Template']
    duration = 20
    
    # Menggunakan regex untuk mencari placeholder '___' dan menggantinya dengan 'if'
    if '___' in question:
        correct_answer = row['correct_answer'] # Hanya menampilkan pengganti dari placeholder
    else:
        correct_answer = ""  # Jika tidak ada placeholder ___

    # Menambahkan ke struktur JSON
    question_entry = {
        "question_id": question_id,
        "material": material,
        "instruction": instruction,
        "question": question,
        "correct_answer": correct_answer,
        "duration": duration
    }
    json_data["quiz"].append(question_entry)

# Menyimpan ke file JSON
output_json_path = 'questions_function.json'
with open(output_json_path, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

print(f"File JSON berhasil disimpan: {output_json_path}")


File JSON berhasil disimpan: questions_function.json
